In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

We're ready to build our first neural network. We will have multiple features we feed into our model, each of which will go through a set of perceptron models to arrive at a response which will be trained to our output.

Like many models we've covered, this can be used as both a regression or classification model.

First, we need to load our dataset. For this example we'll use The Museum of Modern Art in New York's [public dataset](https://media.githubusercontent.com/media/MuseumofModernArt/collection/master/Artworks.csv) on their collection.

In [2]:
artworks = pd.read_csv('../data/Artworks.csv')
#artworks = pd.read_csv('https://media.githubusercontent.com/media/MuseumofModernArt/collection/master/Artworks.csv')

In [3]:
artworks.columns

Index(['Title', 'Artist', 'ConstituentID', 'ArtistBio', 'Nationality',
       'BeginDate', 'EndDate', 'Gender', 'Date', 'Medium', 'Dimensions',
       'CreditLine', 'AccessionNumber', 'Classification', 'Department',
       'DateAcquired', 'Cataloged', 'ObjectID', 'URL', 'ThumbnailURL',
       'Circumference (cm)', 'Depth (cm)', 'Diameter (cm)', 'Height (cm)',
       'Length (cm)', 'Weight (kg)', 'Width (cm)', 'Seat Height (cm)',
       'Duration (sec.)'],
      dtype='object')

We'll also do a bit of data processing and cleaning, selecting columns of interest and converting URL's to booleans indicating whether they are present.

In [4]:
# Select Columns.
artworks = artworks[['Artist', 'Nationality', 'Gender', 'Date', 'Department',
                    'DateAcquired', 'URL', 'ThumbnailURL', 'Height (cm)', 'Width (cm)']]

# Convert URL's to booleans.
artworks['URL'] = artworks['URL'].notnull()
artworks['ThumbnailURL'] = artworks['ThumbnailURL'].notnull()

# Drop films and some other tricky rows.
artworks = artworks[artworks['Department']!='Film']
artworks = artworks[artworks['Department']!='Media and Performance Art']
artworks = artworks[artworks['Department']!='Fluxus Collection']

# Drop missing data.
artworks = artworks.dropna()

In [5]:
artworks.head()

,Artist,Nationality,Gender,Date,Department,DateAcquired,URL,ThumbnailURL,Height (cm),Width (cm)
0,Otto Wagner,(Austrian),(Male),1896,Architecture & Design,1996-04-09,True,True,48.6000,168.9000
1,Christian de Portzamparc,(French),(Male),1987,Architecture & Design,1995-01-17,True,True,40.6401,29.8451
2,Emil Hoppe,(Austrian),(Male),1903,Architecture & Design,1997-01-15,True,True,34.3000,31.8000
3,Bernard Tschumi,(),(Male),1980,Architecture & Design,1995-01-17,True,True,50.8000,50.8000
4,Emil Hoppe,(Austrian),(Male),1903,Architecture & Design,1997-01-15,True,True,38.4000,19.1000


## Building a Model

Now, let's see if we can use multi-layer perceptron modeling (or "MLP") to see if we can classify the department a piece should go into using everything but the department name.

Before we import MLP from SKLearn and establish the model we first have to ensure correct typing for our data and do some other cleaning.

In [6]:
# Get data types.
artworks.dtypes

Artist           object
Nationality      object
Gender           object
Date             object
Department       object
DateAcquired     object
URL                bool
ThumbnailURL       bool
Height (cm)     float64
Width (cm)      float64
dtype: object

The `DateAcquired` column is an object. Let's transform that to a datetime object and add a feature for just the year the artwork was acquired.

In [7]:
artworks['DateAcquired'] = pd.to_datetime(artworks.DateAcquired)
artworks['YearAcquired'] = artworks.DateAcquired.dt.year
artworks['YearAcquired'].dtype

dtype('int64')

Great. Let's do some more miscellaneous cleaning.

In [8]:
# Remove multiple nationalities, genders, and artists.
artworks.loc[artworks['Gender'].str.contains('\) \('), 'Gender'] = '\(multiple_persons\)'
artworks.loc[artworks['Nationality'].str.contains('\) \('), 'Nationality'] = '\(multiple_nationalities\)'
artworks.loc[artworks['Artist'].str.contains(','), 'Artist'] = 'Multiple_Artists'

# Convert dates to start date, cutting down number of distinct examples.
artworks['Date'] = pd.Series(artworks.Date.str.extract(
    '([0-9]{4})', expand=False))[:-1]

# Final column drops and NA drop.
X = artworks.drop(['Department', 'DateAcquired', 'Artist', 'Nationality', 'Date'], 1)

# Create dummies separately.
artists = pd.get_dummies(artworks.Artist)
nationalities = pd.get_dummies(artworks.Nationality)
dates = pd.get_dummies(artworks.Date)

# Concat with other variables, but artists slows this wayyyyy down so we'll keep it out for now
X = pd.get_dummies(X, sparse=True)
X = pd.concat([X, nationalities, dates], axis=1)

Y = artworks.Department

In [9]:
# Alright! We've done our prep, let's build the model.
# Neural networks are hugely computationally intensive.
# This may take several minutes to run.

# Import the model.
from sklearn.neural_network import MLPClassifier

# Establish and fit the model, with a single, 1000 perceptron layer.
mlp = MLPClassifier(hidden_layer_sizes=(100,))
mlp.fit(X, Y)

C:\Users\yfsta\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [10]:
mlp.score(X, Y)

0.7492575033665997

In [11]:
Y.value_counts()/len(Y)

Drawings & Prints        0.622507
Photography              0.226466
Architecture & Design    0.112970
Painting & Sculpture     0.033684
Media and Performance    0.004372
Name: Department, dtype: float64

In [12]:
from sklearn.model_selection import cross_val_score
cross_val_score(mlp, X, Y, cv=5)

C:\Users\yfsta\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


array([0.70095914, 0.73216509, 0.64190186, 0.65473665, 0.58821088])

Now we got a lot of information from all of this. Firstly we can see that the model seems to overfit, though there is still so remaining performance when validated with cross validation. This is a feature of neural networks that aren't given enough data for the number of features present. _Neural networks, in general, like_ a lot _of data_. You may also have noticed something also about neural networks: _they can take a_ long _time to run_. Try increasing the layer size by adding a zero. Feel free to interrupt the kernel if you don't have time...

Also note that we created bools for artist's name but left them out. Both of the above points are the reason for that. It would take much longer to run and it would be much more prone to overfitting.

## Model parameters

Now, before we move on and let you loose with some tasks to work on the model, let's go over the parameters.

We included one parameter: hidden layer size. Remember in the previous lesson, when we talked about layers in a neural network. This tells us how many and how big to make our layers. Pass in a tuple that specifies each layer's size. Our network is 1000 neurons wide and one layer. (100, 4, ) would create a network with two layers, one 100 wide and the other 4.

How many layers to include is determined by two things: computational resources and cross validation searching for convergence. It's generally less than the number of input variables you have.

You can also set an alpha. Neural networks like this use a regularization parameter that penalizes large coefficients just like we discussed in the advanced regression section. Alpha scales that penalty.

Lastly, we'll discuss the activation function. The activation function determines whether the output from an individual perceptron is binary or continuous. By default this is a 'relu', or 'rectified linear unit function' function. In the exercise we went through earlier we used this binary function, but we discussed the _sigmoid_ as a reasonable alternative. The _sigmoid_ (called 'logistic' by SKLearn because it's a 'logistic sigmoid function') allows for continuous variables between 0 and 1, which allows for a more nuanced model. It does come at the cost of increased computational complexity.

If you want to learn more about these, study [activation functions](https://en.wikipedia.org/wiki/Activation_function) and [multilayer perceptrons](https://en.wikipedia.org/wiki/Multilayer_perceptron). The [Deep Learning](http://www.deeplearningbook.org/) book referenced earlier goes into great detail on the linear algebra involved.

You could also just test the models with cross validation. Unless neural networks are your specialty cross validation should be sufficient.

For the other parameters and their defaults, check out the [MLPClassifier documentation](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier).

## Drill: Playing with layers

Now it's your turn. Using the space below, experiment with different hidden layer structures. You can try this on a subset of the data to improve runtime. See how things vary. See what seems to matter the most. Feel free to manipulate other parameters as well. It may also be beneficial to do some real feature selection work...

In [22]:
# Reduce to use 1/10 of data
from sklearn.model_selection import train_test_split
X1, X2, Y1, Y2 = train_test_split(X, Y, test_size=0.1, random_state=0)

Y2.value_counts()/len(Y2)

Drawings & Prints        0.622302
Photography              0.226250
Architecture & Design    0.112433
Painting & Sculpture     0.034588
Media and Performance    0.004427
Name: Department, dtype: float64

In [23]:
# Use 1000 neuron layer as base
mlp2 = MLPClassifier(hidden_layer_sizes=(1000,))
mlp2.fit(X2, Y2)
mlp2.score(X2, Y2)

0.7173953145176166

### Hidden Layer Structure

In [38]:
# Add a hidden layer
mlp2 = MLPClassifier(hidden_layer_sizes=(1000,10))
mlp2.fit(X2, Y2)
mlp2.score(X2, Y2)

0.6223021582733813

Adding a 10 neuron hidden layer decreased accuracy considerably.

In [39]:
# Switch layer order
mlp2 = MLPClassifier(hidden_layer_sizes=(10,1000))
mlp2.fit(X2, Y2)
mlp2.score(X2, Y2)

0.7517985611510791

Switching to a 10 neuron visible layer and 1000 neuron hidden layer resulted in higher accuracy.

In [41]:
# Add additional hidden layer
mlp2 = MLPClassifier(hidden_layer_sizes=(10,1000,10))
mlp2.fit(X2, Y2)
mlp2.score(X2, Y2)

0.6563364692861096

Adding an additional 10 neuron hidden layer reduced accuracy.

In [42]:
# Swap hidden layers
mlp2 = MLPClassifier(hidden_layer_sizes=(10,10,1000))
mlp2.fit(X2, Y2)
mlp2.score(X2, Y2)

0.7002398081534772

Swapping hidden layers increased score some but still resulted in a lower accuracy than the 10/1000 combination.

In [43]:
# Reduce hidden layer size
mlp2 = MLPClassifier(hidden_layer_sizes=(10,10,100))
mlp2.fit(X2, Y2)
mlp2.score(X2, Y2)

0.7415605976757056

Reducing the size of the second hidden layer resulted in an accuracy score just below the 10/1000 combination.

In [44]:
# Reduce hidden layer size
mlp2 = MLPClassifier(hidden_layer_sizes=(10,10,10))
mlp2.fit(X2, Y2)
mlp2.score(X2, Y2)

C:\Users\yfsta\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.7441431470208448

Further reducing the hidden layer size had little impact.

In [46]:
# Add another hidden layer
mlp2 = MLPClassifier(hidden_layer_sizes=(10,10,10,10))
mlp2.fit(X2, Y2)
mlp2.score(X2, Y2)

0.6160302527209002

Adding a third hidden layer significantly reduced accuracy.

In [47]:
# Add another hidden layer
mlp2 = MLPClassifier(hidden_layer_sizes=(10,100,100))
mlp2.fit(X2, Y2)
mlp2.score(X2, Y2)

0.7437742114001107

Increasing hidden layer size from 10 to 100 did not have a significant effect.

In [48]:
# Reduce hidden layer size
mlp2 = MLPClassifier(hidden_layer_sizes=(10,100))
mlp2.fit(X2, Y2)
mlp2.score(X2, Y2)

0.7093709647666482

In [49]:
# Increase hidden layer size
mlp2 = MLPClassifier(hidden_layer_sizes=(10,2000))
mlp2.fit(X2, Y2)
mlp2.score(X2, Y2)

0.7583471684191109

In [51]:
# Increase hidden layer size
mlp2 = MLPClassifier(hidden_layer_sizes=(100,1000))
mlp2.fit(X2, Y2)
mlp2.score(X2, Y2)

0.7261575355100535

#### Results:

| Visible | Hidden 1 | Hidden 2 | Hidden 3 | Accuracy |
|---------|----------|----------|----------|----------|
| 1000    |----------|----------|----------|0.71740   |
| 1000    |10        |----------|----------|0.62230   |
| 10      |1000      |----------|----------|0.75179   |
| 10      |1000      |10        |----------|0.65635   |
| 10      |10        |100       |----------|0.74156   |
| 10      |10        |10        |----------|0.74414   |
| 10      |10        |10        |10        |0.61603   |
| 10      |100       |100       |----------|0.74377   |
| 10      |100       |----------|----------|0.70937   |
| 10      |2000      |----------|----------|0.75835   |
| 100     |1000      |----------|----------|0.72616   |

Of these scenarios, the 10 neuron visible, 2000 neuron hidden layer performed best.

### L2 penalty

In [53]:
mlp2 = MLPClassifier(hidden_layer_sizes=(10,2000),alpha=0.1)
mlp2.fit(X2, Y2)
mlp2.score(X2, Y2)

0.7042980999815532

Increasing L2 penalty caused accuracy to decrease.

In [55]:
mlp2 = MLPClassifier(hidden_layer_sizes=(10,2000),alpha=0.000001)
mlp2.fit(X2, Y2)

0.762589928057554

Decreasing L2 penalty increased accuracy slightly.

### Activation function

In [56]:
mlp2 = MLPClassifier(hidden_layer_sizes=(10,2000),alpha=0.000001,activation='logistic')
mlp2.fit(X2, Y2)
mlp2.score(X2, Y2)

0.6401033019738056

Switching from linear to sigmoid activation function reduced accuracy significantly.

### Cross validation

In [63]:
mlp2 = MLPClassifier(hidden_layer_sizes=(10,2000))
mlp2.fit(X2, Y2)
cross_val_score(mlp2, X2, Y2, cv=5)

C:\Users\yfsta\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


array([0.72718894, 0.73271889, 0.72798525, 0.73650208, 0.6823638 ])

In [65]:
np.std([0.72718894, 0.73271889, 0.72798525, 0.73650208, 0.6823638 ])

0.019783228884225557

Cross validation scores were quite consistent, suggesting a low degree of overfitting.

In [57]:
mlp2 = MLPClassifier(hidden_layer_sizes=(10,2000),alpha=0.000001)
mlp2.fit(X2, Y2)
cross_val_score(mlp2, X2, Y2, cv=5)

array([0.68986175, 0.74423963, 0.73582296, 0.72958006, 0.70544783])

In [62]:
np.std([0.68986175, 0.74423963, 0.73582296, 0.72958006, 0.70544783])

0.020223145767555153

After reducing L2 penalty, there was a slight increase in standard deviation of cross validation scores but results were still fairly consistent.